# Loading data

In [1]:
import json

with open('clinais.train.json') as f:
    data = json.load(f)

from tqdm import tqdm

finalresult = []
for key in tqdm(data['annotated_entries'].keys()):
    ident = data['annotated_entries'][key]['note_id']
    res = []
    tags = []
    gold = data['annotated_entries'][key]['boundary_annotation']['gold']
    currentboundary = ''
    for g in gold:
        res.append(g['span'])
        if(g['boundary'] is None):
            tags.append('I-'+currentboundary)
        else:
            currentboundary = g['boundary']
            tags.append('B-'+currentboundary)
    finalresult.append([ident,res,tags])

# finalresult    

import numpy as np
import itertools
tags = [x[2] for x in finalresult]
tags = np.unique(list(itertools.chain(*tags)))
id2label = {}
label2id = {}
for i,tag in enumerate(tags):
    id2label[i] = tag
    label2id[tag] = i

In [6]:
finalresult = [[x[0],x[1],[label2id[y] for y in x[2]]] for x in finalresult]
#finalresult[0]


In [7]:
with open('clinais.dev.json') as f:
    data = json.load(f)

In [8]:
finalresultdev = []
for key in tqdm(data['annotated_entries'].keys()):
    ident = data['annotated_entries'][key]['note_id']
    res = []
    tags = []
    gold = data['annotated_entries'][key]['boundary_annotation']['gold']
    currentboundary = ''
    for g in gold:
        res.append(g['span'])
        if(g['boundary'] is None):
            tags.append('I-'+currentboundary)
        else:
            currentboundary = g['boundary']
            tags.append('B-'+currentboundary)
    finalresultdev.append([ident,res,tags])

100%|██████████| 127/127 [00:00<00:00, 11283.13it/s]


In [9]:
finalresultdev = [[x[0],x[1],[label2id[y] for y in x[2]]] for x in finalresultdev]


In [10]:
from datasets import Dataset,DatasetDict

In [11]:
import pandas as pd
df = pd.DataFrame(data=finalresult,columns=['id','tokens','tags'])
dataset_train = Dataset.from_pandas(df)

In [12]:
df = pd.DataFrame(data=finalresultdev,columns=['id','tokens','tags'])
dataset_val = Dataset.from_pandas(df)

In [13]:
dataset = DatasetDict(train=dataset_train,val=dataset_val)

# Processing dataset

In [14]:
from transformers import AutoTokenizer

In [15]:
modelCheckpoint = "joheras/roberta-base-biomedical-clinical-es-finetuned-clinais"
tokenizer = AutoTokenizer.from_pretrained(modelCheckpoint)

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/894k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/540k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [16]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [17]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 781
    })
    val: Dataset({
        features: ['id', 'tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 127
    })
})

In [19]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [20]:
import evaluate

seqeval = evaluate.load("seqeval")

In [21]:
import numpy as np



def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [22]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    modelCheckpoint, num_labels=len(id2label), id2label=id2label, label2id=label2id
)

Downloading:   0%|          | 0.00/742 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504M [00:00<?, ?B/s]

Some weights of the model checkpoint at joheras/roberta-base-biomedical-clinical-es-finetuned-clinais were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at joheras/roberta-base-biomedical-clinical-es-finetuned-clinais and are newly initialized: ['classifier.bias', 'classifier.weight

In [23]:
training_args = TrainingArguments(
    output_dir="clinico-roberta-biomedical-finetuned",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Cloning https://huggingface.co/joheras/clinico-roberta-biomedical-finetuned into local empty directory.
The following columns in the training set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: id, tokens, tags. If id, tokens, tags are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
/home/joheras/.local/lib/python3.10/site-packages/transformers/optimization.py:346: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 781
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2500
  Number of trainable parameters = 125398286


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/grupoa/config/miniconda3/envs/fastai/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.219861,0.003287,0.005263,0.004047,0.575595
2,No log,0.730559,0.203074,0.264211,0.229643,0.802140
3,No log,0.636648,0.296721,0.381053,0.333641,0.823541
4,No log,0.613526,0.349684,0.465263,0.399277,0.830374
5,No log,0.584473,0.342063,0.453684,0.390045,0.833112
6,No log,0.569699,0.330709,0.442105,0.378378,0.839041
7,No log,0.541544,0.321053,0.449474,0.374561,0.847059
8,No log,0.542955,0.358862,0.517895,0.423955,0.856659
9,No log,0.551321,0.334190,0.547368,0.415004,0.860357
10,No log,0.568054,0.376891,0.576842,0.455907,0.858212


The following columns in the evaluation set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: id, tokens, tags. If id, tokens, tags are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 127
  Batch size = 32
/grupoa/config/miniconda3/envs/fastai/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to clinico-roberta-biomedical-finetuned/checkpoint-25
Configuration saved in clinico-roberta-biomedical-finetuned/checkpoint-25/config.json
Model weights saved in clinico-roberta-biomedical-finetuned/checkpoint-25/pytorch_model.bin
tokenizer config file saved in clinico-roberta-biomedical-finetuned/

TrainOutput(global_step=2500, training_loss=0.0791772575378418, metrics={'train_runtime': 2030.3094, 'train_samples_per_second': 38.467, 'train_steps_per_second': 1.231, 'total_flos': 2.04094907123712e+16, 'train_loss': 0.0791772575378418, 'epoch': 100.0})

In [24]:
trainer.push_to_hub()

Saving model checkpoint to clinico-roberta-biomedical-finetuned
Configuration saved in clinico-roberta-biomedical-finetuned/config.json
Model weights saved in clinico-roberta-biomedical-finetuned/pytorch_model.bin
tokenizer config file saved in clinico-roberta-biomedical-finetuned/tokenizer_config.json
Special tokens file saved in clinico-roberta-biomedical-finetuned/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/478M [00:00<?, ?B/s]

Upload file runs/Mar15_11-30-21_minion/events.out.tfevents.1678876229.minion.3939000.0:  62%|######1   | 32.0k…

To https://huggingface.co/joheras/clinico-roberta-biomedical-finetuned
   63167ae..ed2886d  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.5095435684647303}, {'name': 'Recall', 'type': 'recall', 'value': 0.6463157894736842}, {'name': 'F1', 'type': 'f1', 'value': 0.5698375870069605}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.8623337756571332}]}
To https://huggingface.co/joheras/clinico-roberta-biomedical-finetuned
   ed2886d..6bc7ff1  main -> main



'https://huggingface.co/joheras/clinico-roberta-biomedical-finetuned/commit/ed2886ddf8bb117304adc5edc5fa107afb4f0398'

In [25]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Wed Mar 15 12:15:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.76       Driver Version: 515.76       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| 30